# Simple retirement analysis

The goal of this notebook is to get a baseline for how much I need to be saving to provide a comfortable retirement.  I would like to aim for retirement by 60, both for a simple number and a conservative estimate in case I'm not prepared by then.

## Imports

In [1]:
import math

## Expected required income

### Assumptions for the increased cost of living in 30 years

I'm performing a pretty naive analysis based on wage growth 30 years ago vs. today.  A Google search suggests that the median income in 1995 was $34,000, whereas in 2025 it's stated as $62,000.  The household median income in 1995 was $34,000 as welll (suspiciously), whereas in 2025 it's $84,000.  The _average_ household income in 1995 was $45,000, whereas that figure is $121,000 in 2025.  The ratios are evaluated below.

In [12]:
median_individual_scale = 62/34
print("2025/1995 median income: ", median_individual_scale)
median_household_scale = 84/34
print("2025/1995 median household income: ", median_household_scale)
mean_household_scale = 121/45
print("2025/1995 mean household income: ", mean_household_scale)

2025/1995 median income:  1.8235294117647058
2025/1995 median household income:  2.4705882352941178
2025/1995 mean household income:  2.688888888888889


This provides a range of income multipliers for retirement age as roughly 1.8-2.7 times expected required income in present-day dollars.

### Assumptions for retiree expenses

#### Cost-based estimate

I'm thinking _optimistically_ that household expenses might be similar to how they are now with the savings included as expenses.  Using convenient estimates, my rough monthly expenses include the following.
- $2,000 rent
- $600 in groceries
- $1,200 in savings
- $400 in misc
I.e., around $4,200/month.  I'm sure Hailey's got additional expenses I'm missing, so let's bump things up to an even $5,000/month, or $60,000/year.  Notably this is after tax, and this already feels quite low, so I'm going to say we need to aim for **$100,000** in taxable income for retirement (assuming my roth IRA doesn't dominate my eventual retirement accounts).

#### Present-day income-based estimate

Because $100,000/year in present-day dollars feels low for a 60 year old couple, I'm going to also consider our current household income as another baseline.  [_Note: I do think you accumulate assets by your 60s--cars, home, furniture, etc.--so this may not be as crazy as it sounds if costs are mostly maintenance vs. big purchases._]  To pick an even number, I'll put the mark at **$250,000**.

### Summary

I'm starting with two kinds of baseline estimates for required income in retirement using 2025 dollars, and then use a range of scale factors to approximate my income requirements in 2055 dollars.

#### Estimates in 2025 dollars

In [13]:
cost_estimate = 100_000
print("Income estimate in 2025 dollars based on cost: ", cost_estimate)
household_estimate = 250_000
print("Income estimate in 2025 dollars based on current income: ", household_estimate)

Income estimate in 2025 dollars based on cost:  100000
Income estimate in 2025 dollars based on current income:  250000


#### Cost-based projections in 2055 dollars

In [14]:
print("Scaled by median individual income: ",
      median_individual_scale * cost_estimate)
print("Scaled by median household income: ",
      median_household_scale * cost_estimate)
print("Scaled by mean household income: ",
      mean_household_scale * cost_estimate)

Scaled by median individual income:  182352.94117647057
Scaled by median household income:  247058.82352941178
Scaled by mean household income:  268888.88888888893


#### Present-day-based projections in 2055 dollars

In [15]:
print("Scaled by median individual income: ",
      median_individual_scale * household_estimate)
print("Scaled by median household income: ",
      median_household_scale * household_estimate)
print("Scaled by mean household income: ",
      mean_household_scale * household_estimate)

Scaled by median individual income:  455882.35294117645
Scaled by median household income:  617647.0588235294
Scaled by mean household income:  672222.2222222222


## Expected required 

## Expected required future savings

### Estimated investment returns

A google search shows many resources claiming a 5-7% annual return is typical in a 401k portfolio.  This sounds low, but maybe it includes a transition to hedging years before retirement.  

In [3]:
rate_low = 0.05
print("Lower estimated annual rate of return: ", rate_low)
rate_high = 0.07
print("Higher estimated annnual rate of return: ", rate_high)

Lower estimated annual rate of return:  0.05
Higher estimated annnual rate of return:  0.07


## Expected savings rate based on future savings targets

### The compounding analysis

For now, I'm going to consider a constant rate of savings to simplify the math.  This is definitely an over-simplification, but I think it still has utility so long as I can tune the following.
- the period of investment
- the pricipal
- the target amount at retirement (investment maturity)
- the expected typical rate of return (input _per annum_, but internally compounded _monthly_)

The value $V$ of an investment with principal $P$ over the period $T$ (in years) can be related to the (currently constant) savings rate $s$ (in $/month),  
$V = P \left(1 + \frac{r_M}{12}\right)^{12T}+ \sum_{i=1}^{12 T} s \left(1+\frac{r_M}{12}\right)^{i} \quad ,$  
where $r_M$ is the rate of return compounded monthly, determined from the typical rate of return _per annum_ $r$ via  
$r_M = 12\left[\left(1 + r\right)^{1/12} - 1\right] \quad .$  
Notably, this assumes that we can ignore edge effects by rounding $T$ to an integer number of years.[^At the time of writing, I'm taking the floor of $T$ as the integral number of years.]  

### The future value predicted by savings rate and principal

Since it's a useful calculation to have on-hand, this function calculates the future value vased on a savings rate, investment period, and expected rate of return _per annum_.

####

In [ ]:
def annual_rate_to_monthly(annual_rate: float) -> float:
    """
    ### Converts an annual compounding rate to the monthly compounding rate
    - annual_rate - the rate of return when compounded annually
    """
    return 12 * (math.pow(1 + annual_rate, 1/12) - 1)

def future_value(principal: float, period: int, annual_rate: float, savings_rate: float) -> float:
    """
    ### Calculates the future value (in dollars) of a savings investment strategy.  
    - principal - initial investment in dollars  
    - period - number of years to predict to the value of this investment  
    - annual_rate - the expected rate of return per year  
    - savings_rate - the monthly contribution to the investment in dollars
    """
    monthly_rate = annual_rate_to_monthly(annual_rate)
    compounding_factor = 1 + monthly_rate / 12
    future_principal = principal * math.pow(compounding_factor, 12 * period)
    future_savings = savings_rate * sum(math.pow(compounding_factor, i) for i in range(1, 12 * period + 1))
    return future_principal + future_savings

### The required savings rate for a desired future value

Because I've assumed a constant rate of savings $s$, backing out the implied savings rate for a fixed principal $P$ and desired future value $V$ is simple algebra.  
#### $s = \frac{V - P\left(1 + \frac{r_M}{12}\right)^{12T}}{\sum_{i=1}^{12 T} \left(1+\frac{r_M}{12}\right)^{i}} \quad$

In [29]:
def required_monthly_savings(future_value: float, principal: float, period: int, annual_rate: float) -> float:
    """
    ### Calculates the required monthly savings rate (in dollars) to achieve the desired future value (in dollars).
    - future_value - the desired future value in dollars
    - principal - the initial investment in dollars
    - period - the number of years where the savings rate is contributed monthly
    - annual_rate - the expected rate of return per year
    """
    monthly_rate = annual_rate_to_monthly(annual_rate)
    compounding_factor = 1 + monthly_rate / 12
    future_principal = principal * math.pow(compounding_factor, 12 * period)
    future_savings_factor = sum(math.pow(compounding_factor, i) for i in range(1, 12 * period + 1))
    return (future_value - future_principal) / future_savings_factor

Todo - return after target future savings values are determined